In [ ]:
from datetime import date
from pathlib import Path

import ipywidgets as widgets
import yaml
from ipyfilechooser import FileChooser
from matplotlib import cm

from src.utils import unique_session_id
from src.viirs import APPEEARSDownloader, process_products
from src.visualisation import product_visualiser

# Session Setup
Please run the following cells to setup your session. (i.e. Setting up appEEARS task name, download and processing directories)

You may update the task name below from the default value to your preferred value.

**Note**: Your task name MUST be unique. You may wish to visit https://appeears.earthdatacloud.nasa.gov/ periodically to remove old requests.

In [ ]:
task_name = widgets.Text(description="Task Name: ", value=unique_session_id())
display(task_name)

In [ ]:
# Load Config File
with open(file="config.yaml", mode="r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

In [ ]:
# Load Directory Paths and Create Directories
raw_dir = Path(config["VIIRS_RAW"], task_name.value)
processed_dir = Path(config["VIIRS_PROCESSED"], task_name.value)

raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

# Submit a request for VIIRS Products

Please enter your APPEEARS login credentials.

**Note**: You will need to create an account on https://appeears.earthdatacloud.nasa.gov/ before proceeding.

In [ ]:
username = widgets.Text(description="APPEEARS Username: ")
username.style.description_width = "initial"

password = widgets.Password(description="APPEEARS Password: ")
password.style.description_width = "initial"

credential_widgets = widgets.VBox([username, password])
display(credential_widgets)

In [ ]:
# Initialize APPEARS Downloader and Login
appeears = APPEEARSDownloader(username.value, password.value)
token = appeears.get_token()

Please enter your product search criteria below.

In [ ]:
start_date = widgets.DatePicker(description="Start Date: ", value=date(2024, 1, 1))
end_date = widgets.DatePicker(description="End Date: ", value=date(2024, 1, 8))

min_lon = widgets.FloatText(description="Min Lon: ", value=116.01897)
max_lon = widgets.FloatText(description="Max Lon: ", value=116.20093)
min_lat = widgets.FloatText(description="Min Lat: ", value=-32.30959)
max_lat = widgets.FloatText(description="Max Lat: ", value=-31.98176)
shapefile_path = FileChooser(
    title="Upload a Shapefile (Optional): ", filter_pattern="*.shp"
)

product = widgets.Select(
    options=["VNP09GA.001", "VNP09GA.002"],
    value="VNP09GA.001",
    rows=2,
    description="VIIRS Product: ",
)
product.style.description_width = "initial"

search_widgets = widgets.VBox(
    [
        start_date,
        end_date,
        widgets.HTML("<br>"),
        widgets.Label(
            "Please enter your area of interest coordinates or upload a Shapefile."
        ),
        min_lon,
        max_lon,
        min_lat,
        max_lat,
        shapefile_path,
        widgets.HTML("<br>"),
        product,
    ]
)
display(search_widgets)

Depending on your search parameters, APPEEARS may take a while to process your request.

More information on VIIRS filenames can be found [here](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/s-npp-nasa-viirs-overview/).

In [ ]:
# Generate Task for APPEEARS
task = appeears.generate_task(
    task_name=task_name.value,
    min_lat=min_lat.value,
    min_lon=min_lon.value,
    max_lat=max_lat.value,
    max_lon=max_lon.value,
    shapefile=shapefile_path.selected,
    start_date=start_date.value,
    end_date=end_date.value,
    product_type=product.value,
)

# Submit Task to APPEEARS and Retrieve Task ID
task_id = appeears.submit_task(task=task, token=token)

In [ ]:
# Check submitted task status
status = appeears.check_task(task_id=task_id, token=token, timeout=120)

In [ ]:
# Download satellite product from APPEARS
if status:
    appeears.download(
        task_name=task_name.value,
        task_id=task_id,
        product_type=product.value,
        download_dir=raw_dir,
        token=token,
    )

In [ ]:
# Logout from APPEARS API
appeears.logout(token=token)

# NDVI, NDMI and NBR Processing

Please select which VIIRS product to process. (Use Ctrl to select multiple).

In [ ]:
viirs_dirs = [
    dir
    for dir in list(Path(raw_dir, "VNP09GA.001").glob("*"))
    + list(Path(raw_dir, "VNP09GA.002").glob("*"))
    if dir.is_dir()
]

selected_products = widgets.SelectMultiple(
    options=viirs_dirs,
    rows=10,
    description="Select a VIIRS product to process: ",
    layout=widgets.Layout(width="auto"),
)
selected_products.style.description_width = "initial"

select_all_products = widgets.Checkbox(
    value=False, description="Select all products? ", indent=False
)
processing_widgets = widgets.VBox([selected_products, select_all_products])

display(processing_widgets)

In [ ]:
process_products(
    product_paths=selected_products.value,
    viirs_dirs=viirs_dirs,
    viirs_processed_dir=processed_dir,
    all_products=select_all_products.value,
)

# Visualisation

Please select a processed VIIRS product to visualise via ipyleaflet. (Use Ctrl to select multiple).

You have the option to select a colormap for your raster layers.

Note: You WILL need to run the %%javascript cell below before displaying the map.

In [ ]:
processed_dirs = [dir for dir in list(processed_dir.glob("*")) if dir.is_dir()]

selected_products = widgets.SelectMultiple(
    options=processed_dirs,
    rows=10,
    description="Select a VIIRS product to visualise: ",
    layout=widgets.Layout(width="auto"),
)
selected_products.style.description_width = "initial"

selected_resample = widgets.Checkbox(
    value=True, 
    description="(RECOMMENDED) Apply a downscaling factor to reduce the spatial resolution of product for rapid display?", 
    indent=False,
    layout=widgets.Layout(width="auto"),
)


resampling_factor = widgets.IntText(
    value=6, description="Downscaling Factor: ", disabled=True
)
resampling_factor.style.description_width = "initial"

description_box = widgets.HTML(
    value="It is recommended to do any actual analysis in python outside of this notebook or in a GIS program such as QGIS/ArcGIS",
    placeholder='',
    description='Note:',
)

selected_colormap = widgets.RadioButtons(
    options={"Viridis": cm.viridis, "Jet": cm.jet, "Turbo": cm.turbo, },
    description="Select a colormap: ",
    layout=widgets.Layout(width="auto"),
)

selected_percentile = widgets.Checkbox(
    value=False, 
    description="Apply Histogram-filter to colormap, potentially enhancing contrast and improve image display?", 
    indent=False,
    layout=widgets.Layout(width="auto"),
)

colormap_fname = 'notebook_images/colormap_preview.png'
with open(colormap_fname, 'rb') as file:
    image = file.read()
colormap_image_widget = widgets.Image(
    value=image,
    format='png',
    width=600
)

visualisation_widgets = widgets.VBox(
    [
        selected_products,
        selected_resample,
        resampling_factor,
        description_box,
        selected_colormap,
        selected_percentile,
        colormap_image_widget
    ]
)
display(visualisation_widgets)

In [ ]:
%%javascript
// Check that we havent already applied the fix. If we havent, apply the fix.
if (window.pixFix === undefined) { 
 window.pixFix = document.createElement('style');
 window.pixFix.innerText = '.leaflet-image-layer { image-rendering: pixelated }';
 console.log(window.pixFix)
 document.head.appendChild(window.pixFix);
}

In [ ]:
product_visualiser(
    product_paths=selected_products.value,
    product_type="VIIRS",
    apply_resample=False,
    resample_factor=1,
    use_percentile=selected_percentile.value,
    cmap=selected_colormap.value,
)